In [9]:
# Dependencies and Setup
# render the figure in a notebook instead of displaying a dump of the figure object.
%matplotlib inline  
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

#mouse data : Mouse ID	Drug
mouseData_df= pd.read_csv("data/mouse_drug_data.csv")
#trial data: Mouse ID	Timepoint	Tumor Volume (mm3)	Metastatic Sites
trialData_df= pd.read_csv("data/clinicaltrial_data.csv")

In [38]:
allData_df=pd.merge(mouseData_df,trialData_df, on=["Mouse ID","Mouse ID"])
allData_df.head(3)

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0


In [47]:
#mean Tumor volume data by drug and timepoint
#allData_df.set_index(['Drug'])
#gdata.head()
 
gdata= allData_df.groupby(['Drug',"Timepoint"])
gdata_df= pd.DataFrame({"Drug":gdata["Drug"], "Timepoint":gdata["Timepoint"], "Tumor Vol":gdata["Tumor Volume (mm3)"].mean()})
gdata_df= pd.DataFrame( {"Tumor Vol":gdata["Tumor Volume (mm3)"].mean()})
gdata_df.reset_index()
gdata_df.head(12)

Tumor Vol
Drug      Timepoint           
Capomulin 0          45.000000
          5          44.266086
          10         43.084291
          15         42.064317
          20         40.716325
          25         39.939528
          30         38.769339
          35         37.816839
          40         36.958001
          45         36.236114
Ceftamin  0          45.000000
          5          46.503051